# 计算WP和DMC_620方法的读写带宽

先度量一下单个文件吧

In [1]:
repoPath = "/home/eerqi/Workplace/mem_bw_exp/repo/"
csv_path_DMC = repoPath + 'data/csv/dmc_620/500.perlbench_r_iter0_1732726637.csv'
csv_path_WC = repoPath + 'data/csv/watch_point/500.perlbench_r_iter0_1732727831.csv'

## 预处理

从 `perf stat` 输出的 CSV 文件中提取数据，形成数据表

- timestamp 时间戳，每秒一次
- event 事件名
- value 事件计数值

In [2]:
import pandas as pd
import numpy as np

处理 event，从 event 中提取出：type 读写类型（read or write），device_id 即 dmc620 在 /sys/devices/ 的编号，socket 即 dmc620 设备所属的 socket

## 分析


然后我们来处理dmc的

In [3]:

perf_raw_data_DMC = pd.read_csv(csv_path_DMC,
                            sep='|',
                            header=None, 
                            names=["timestamp", "value", "event"], 
                            usecols=[0, 1, 3])

perf_raw_data_DMC

,timestamp,value,event
0,1.000256,60398656,"arm_dmc620_10008c000/clkdiv2_rdwr,mask=1,match=0/"
1,1.000256,60061416,"arm_dmc620_10008c400/clkdiv2_rdwr,mask=1,match=0/"
2,1.000256,60048261,"arm_dmc620_10008c800/clkdiv2_rdwr,mask=1,match=0/"
3,1.000256,60544975,"arm_dmc620_10008cc00/clkdiv2_rdwr,mask=1,match=0/"
4,1.000256,60317585,"arm_dmc620_10008d000/clkdiv2_rdwr,mask=1,match=0/"
...,...,...,...
7259,226.541443,231512,"arm_dmc620_50008cc00/clkdiv2_rdwr,mask=1,match=1/"
7260,226.541443,231025,"arm_dmc620_50008d000/clkdiv2_rdwr,mask=1,match=1/"
7261,226.541443,231016,"arm_dmc620_50008d400/clkdiv2_rdwr,mask=1,match=1/"
7262,226.541443,230999,"arm_dmc620_50008d800/clkdiv2_rdwr,mask=1,match=1/"


In [4]:
perf_agg_data_DMC = perf_raw_data_DMC.groupby(['event']).agg(value=('value', 'sum'), timestamp = ('timestamp', 'max')).reset_index()

print(perf_agg_data_DMC)

                                                event        value   timestamp
0   arm_dmc620_10008c000/clkdiv2_rdwr,mask=1,match=0/  10585509376  226.541443
1   arm_dmc620_10008c000/clkdiv2_rdwr,mask=1,match=1/   1585254638  226.541443
2   arm_dmc620_10008c400/clkdiv2_rdwr,mask=1,match=0/  10588140447  226.541443
3   arm_dmc620_10008c400/clkdiv2_rdwr,mask=1,match=1/   1588219286  226.541443
4   arm_dmc620_10008c800/clkdiv2_rdwr,mask=1,match=0/  10583260166  226.541443
5   arm_dmc620_10008c800/clkdiv2_rdwr,mask=1,match=1/   1591191059  226.541443
6   arm_dmc620_10008cc00/clkdiv2_rdwr,mask=1,match=0/  10559556047  226.541443
7   arm_dmc620_10008cc00/clkdiv2_rdwr,mask=1,match=1/   1598709819  226.541443
8   arm_dmc620_10008d000/clkdiv2_rdwr,mask=1,match=0/  10594683908  226.541443
9   arm_dmc620_10008d000/clkdiv2_rdwr,mask=1,match=1/   1582831996  226.541443
10  arm_dmc620_10008d400/clkdiv2_rdwr,mask=1,match=0/  10560763469  226.541443
11  arm_dmc620_10008d400/clkdiv2_rdwr,mask=1,match=1

In [5]:
perf_agg_data_DMC['type'] = perf_agg_data_DMC['event'].apply(
    lambda x: 'read' if x[-2:-1] == '0' else 'write'
)
perf_agg_data_DMC['device_id'] = perf_agg_data_DMC['event'].apply(
    lambda x: x[11:20]
)
perf_agg_data_DMC['socket'] = perf_agg_data_DMC['device_id'].apply(
    lambda x: 0 if x[0] == '1' else '1'
)

perf_agg_data_DMC = perf_agg_data_DMC.drop('event', axis=1)

perf_agg_data_DMC = perf_agg_data_DMC[perf_agg_data_DMC["type"] == "write"]
perf_agg_data_DMC

,value,timestamp,type,device_id,socket
1,1585254638,226.541443,write,10008c000,0
3,1588219286,226.541443,write,10008c400,0
5,1591191059,226.541443,write,10008c800,0
7,1598709819,226.541443,write,10008cc00,0
9,1582831996,226.541443,write,10008d000,0
11,1586775801,226.541443,write,10008d400,0
13,1596843987,226.541443,write,10008d800,0
15,1588950345,226.541443,write,10008dc00,0
17,1591677306,226.541443,write,50008c000,1
19,1597430163,226.541443,write,50008c400,1


In [6]:
bw_all_DMC = perf_agg_data_DMC.groupby(['timestamp']).agg(value=('value', 'sum')).reset_index()

bw_all_DMC['interval'] = bw_all_DMC['timestamp'].diff()
bw_all_DMC.loc[0, 'interval'] = bw_all_DMC.loc[0, 'timestamp']

bw_all_DMC['memory bandwidth (GB/s)'] = (bw_all_DMC['value'] * 64) / (bw_all_DMC['interval'] * 1000000000)

bw_all_DMC

,timestamp,value,interval,memory bandwidth (GB/s)
0,226.541443,25456969783,226.541443,7.191823


In [7]:
def dmc_620_avg(perf_raw_data_DMC, read_or_write = "read"):
    perf_agg_data_DMC = perf_raw_data_DMC.groupby(['event']).agg(value=('value', 'sum'), timestamp = ('timestamp', 'max')).reset_index()
    perf_agg_data_DMC['type'] = perf_agg_data_DMC['event'].apply(
        lambda x: 'read' if x[-2:-1] == '0' else 'write'
    )
    perf_agg_data_DMC['device_id'] = perf_agg_data_DMC['event'].apply(
        lambda x: x[11:20]
    )
    perf_agg_data_DMC['socket'] = perf_agg_data_DMC['device_id'].apply(
        lambda x: 0 if x[0] == '1' else '1'
    )

    perf_agg_data_DMC = perf_agg_data_DMC[perf_agg_data_DMC["type"] == read_or_write]

    perf_agg_data_DMC = perf_agg_data_DMC.drop('event', axis=1)

    bw_all_DMC = perf_agg_data_DMC.groupby(['timestamp']).agg(value=('value', 'sum')).reset_index()

    bw_all_DMC['interval'] = bw_all_DMC['timestamp'].diff()
    bw_all_DMC.loc[0, 'interval'] = bw_all_DMC.loc[0, 'timestamp']

    bw_all_DMC['memory bandwidth (GB/s)'] = (bw_all_DMC['value'] * 64) / (bw_all_DMC['interval'] * 1000000000)

    return bw_all_DMC['memory bandwidth (GB/s)'][0]

In [8]:
print(dmc_620_avg(perf_raw_data_DMC, "read"))
print(dmc_620_avg(perf_raw_data_DMC, "write"))

49.33133554892101
7.191823460600796


然后把watch-point方法的平均值也求上一求

In [9]:
import pandas as pd
import numpy as np

perf_raw_data_WC = pd.read_csv(csv_path_WC,
                            sep='|',
                            header=None, 
                            names=["timestamp", "value", "event"], 
                            usecols=[0, 1, 3])

print(perf_raw_data_WC)

       timestamp      value                                              event
0       1.001156  115774015  arm_cmn_0/watchpoint_up,bynodeid=1,nodeid=0x8,...
1       1.001156  114415098  arm_cmn_0/watchpoint_up,bynodeid=1,nodeid=0x10...
2       1.001156  112946130  arm_cmn_0/watchpoint_up,bynodeid=1,nodeid=0x18...
3       1.001156  112238045  arm_cmn_0/watchpoint_up,bynodeid=1,nodeid=0x20...
4       1.001156  111628945  arm_cmn_0/watchpoint_up,bynodeid=1,nodeid=0x1c...
...          ...        ...                                                ...
7003  219.001273   16785324  arm_cmn_1/watchpoint_down,bynodeid=1,nodeid=0x...
7004  219.001273   16772100  arm_cmn_1/watchpoint_down,bynodeid=1,nodeid=0x...
7005  219.001273   16580736  arm_cmn_1/watchpoint_down,bynodeid=1,nodeid=0x...
7006  219.001273   16600376  arm_cmn_1/watchpoint_down,bynodeid=1,nodeid=0x...
7007  219.001273   16332575  arm_cmn_1/watchpoint_down,bynodeid=1,nodeid=0x...

[7008 rows x 3 columns]


In [10]:
perf_agg_data_WC = perf_raw_data_WC.groupby(['event']).agg(value=('value', 'sum'), timestamp = ('timestamp', 'max')).reset_index()

perf_agg_data_WC

,event,value,timestamp
0,"arm_cmn_0/watchpoint_down,bynodeid=1,nodeid=0x...",3037465424,219.001273
1,"arm_cmn_0/watchpoint_down,bynodeid=1,nodeid=0x...",3040579022,219.001273
2,"arm_cmn_0/watchpoint_down,bynodeid=1,nodeid=0x...",3040477152,219.001273
3,"arm_cmn_0/watchpoint_down,bynodeid=1,nodeid=0x...",3044037456,219.001273
4,"arm_cmn_0/watchpoint_down,bynodeid=1,nodeid=0x...",3046276947,219.001273
5,"arm_cmn_0/watchpoint_down,bynodeid=1,nodeid=0x...",3039501646,219.001273
6,"arm_cmn_0/watchpoint_down,bynodeid=1,nodeid=0x...",3067159002,219.001273
7,"arm_cmn_0/watchpoint_down,bynodeid=1,nodeid=0x...",3037192930,219.001273
8,"arm_cmn_0/watchpoint_up,bynodeid=1,nodeid=0x10...",19736827274,219.001273
9,"arm_cmn_0/watchpoint_up,bynodeid=1,nodeid=0x18...",19690128329,219.001273


In [11]:
perf_agg_data_WC['type'] = perf_agg_data_WC['event'].apply(
    lambda x: 'read' if x[-2:-1] == '0' else 'write'
)
perf_agg_data_WC['device_id'] = perf_agg_data_WC['event'].apply(
    lambda x: x[11:20]
)
perf_agg_data_WC['socket'] = perf_agg_data_WC['device_id'].apply(
    lambda x: 0 if x[0] == '1' else '1'
)

perf_agg_data_WC = perf_agg_data_WC[perf_agg_data_WC["event"].str.contains("watchpoint_up")]

perf_agg_data_WC = perf_agg_data_WC.drop('event', axis=1)
perf_agg_data_WC

,value,timestamp,type,device_id,socket
8,19736827274,219.001273,write,atchpoint,1
9,19690128329,219.001273,write,atchpoint,1
10,19668086748,219.001273,write,atchpoint,1
11,19634793145,219.001273,write,atchpoint,1
12,19646782602,219.001273,write,atchpoint,1
13,19641444648,219.001273,write,atchpoint,1
14,19691928159,219.001273,write,atchpoint,1
15,19740340949,219.001273,write,atchpoint,1
24,21473405919,219.001273,write,atchpoint,1
25,21455482699,219.001273,write,atchpoint,1


In [12]:
bw_all_WC = perf_agg_data_WC.groupby(['timestamp']).agg(value=('value', 'sum')).reset_index()

bw_all_WC['interval'] = bw_all_WC['timestamp'].diff()
bw_all_WC.loc[0, 'interval'] = bw_all_WC.loc[0, 'timestamp']

bw_all_WC['memory bandwidth (GB/s)'] = (bw_all_WC['value'] * 64) / (bw_all_WC['interval'] * 1000000000)


In [13]:
bw_all_WC['memory bandwidth (GB/s)'][0]

np.float64(96.25653018808727)

In [14]:
def watch_point_avg(perf_raw_data_WC, up_or_down = "up"):
    perf_agg_data_WC = perf_raw_data_WC.groupby(['event']).agg(value=('value', 'sum'), timestamp = ('timestamp', 'max')).reset_index()
    perf_agg_data_WC = perf_agg_data_WC[perf_agg_data_WC["event"].str.contains("watchpoint_" + up_or_down)]
    perf_agg_data_WC = perf_agg_data_WC.drop('event', axis=1)

    bw_all_WC = perf_agg_data_WC.groupby(['timestamp']).agg(value=('value', 'sum')).reset_index()

    bw_all_WC['interval'] = bw_all_WC['timestamp'].diff()
    bw_all_WC.loc[0, 'interval'] = bw_all_WC.loc[0, 'timestamp']

    bw_all_WC['memory bandwidth (GB/s)'] = (bw_all_WC['value'] * 32) / (bw_all_WC['interval'] * 1000000000)

    return bw_all_WC['memory bandwidth (GB/s)'][0]

In [15]:
print(watch_point_avg(perf_raw_data_WC))
print(watch_point_avg(perf_raw_data_WC, "down"))

48.128265094043634
7.165274441939026


现在要做的事情就是把所有的平均值都给他算一遍吧.首先我们要学会怎么把一个benchmark的几段数据合成一段。

In [17]:
import os

DMC_csv_dir = repoPath + "data/csv/dmc_620/"
WP_csv_dir = repoPath + "data/csv/watch_point/"


DMC_files = os.listdir(DMC_csv_dir)
print(DMC_files)

WP_files = os.listdir(WP_csv_dir)
print(WP_files)

['548.exchange2_r_iter0_1732750002.csv', '527.cam4_r_iter0_1732794948.csv', '510.parest_r_iter0_1732768188.csv', '500.perlbench_r_iter0_1732726864.csv', '500.perlbench_r_iter0_1732726637.csv', '554.roms_r_iter0_1732815649.csv', '503.bwaves_r_iter0_1732756430.csv', '502.gcc_r_iter0_1732729463.csv', '503.bwaves_r_iter0_1732756031.csv', '511.povray_r_iter0_1732775012.csv', '502.gcc_r_iter0_1732729258.csv', '544.nab_r_iter0_1732800100.csv', '525.x264_r_iter0_1732744989.csv', '508.namd_r_iter0_1732765192.csv', '503.bwaves_r_iter0_1732757065.csv', '507.cactuBSSN_r_iter0_1732763198.csv', '538.imagick_r_iter0_1732797687.csv', '502.gcc_r_iter0_1732729345.csv', '541.leela_r_iter0_1732747999.csv', '500.perlbench_r_iter0_1732726973.csv', '503.bwaves_r_iter0_1732757570.csv', '520.omnetpp_r_iter0_1732738175.csv', '557.xz_r_iter0_1732752354.csv', '526.blender_r_iter0_1732792925.csv', '519.lbm_r_iter0_1732778962.csv', '505.mcf_r_iter0_1732732987.csv', '502.gcc_r_iter0_1732729593.csv', '531.deepsjeng_r

In [18]:
DMC_bench = [file for file in DMC_files if file.startswith("500")]
WP_bench = [file for file in WP_files if file.startswith("500")]

print(DMC_bench)
print(WP_bench)

['500.perlbench_r_iter0_1732726864.csv', '500.perlbench_r_iter0_1732726637.csv', '500.perlbench_r_iter0_1732726973.csv']
['500.perlbench_r_iter0_1732728160.csv', '500.perlbench_r_iter0_1732728050.csv', '500.perlbench_r_iter0_1732727831.csv']


In [19]:
WP_datas = []
for file in WP_bench:
    perf_raw_data_WP = pd.read_csv(WP_csv_dir + file,
                                sep='|',
                                header=None, 
                                names=["timestamp", "value", "event"], 
                                usecols=[0, 1, 3])
    perf_agg_data_WP = perf_raw_data_WP.groupby(['event']).agg(value=('value', 'sum'), timestamp = ('timestamp', 'max')).reset_index()
    WP_datas.append(perf_agg_data_WP)

# for data in WP_datas:
#     print(len(data))

df_concatenated = pd.concat(WP_datas, axis=0)

bench_perf_data = df_concatenated.groupby(['event']).agg(value=('value', 'sum'), timestamp = ('timestamp', 'sum')).reset_index()

print(watch_point_avg(bench_perf_data, "up"))

54.64412694846594


In [20]:
def dmc_620_avg(perf_raw_data_DMC, read_or_write = "write"):
    perf_agg_data_DMC = perf_raw_data_DMC.groupby(['event']).agg(value=('value', 'sum'), timestamp = ('timestamp', 'max')).reset_index()
    perf_agg_data_DMC['type'] = perf_agg_data_DMC['event'].apply(
        lambda x: 'read' if x[-2:-1] == '0' else 'write'
    )
    perf_agg_data_DMC['device_id'] = perf_agg_data_DMC['event'].apply(
        lambda x: x[11:20]
    )
    perf_agg_data_DMC['socket'] = perf_agg_data_DMC['device_id'].apply(
        lambda x: 0 if x[0] == '1' else '1'
    )

    perf_agg_data_DMC = perf_agg_data_DMC[perf_agg_data_DMC["type"] == read_or_write]

    perf_agg_data_DMC = perf_agg_data_DMC.drop('event', axis=1)

    bw_all_DMC = perf_agg_data_DMC.groupby(['timestamp']).agg(value=('value', 'sum')).reset_index()

    bw_all_DMC['interval'] = bw_all_DMC['timestamp'].diff()
    bw_all_DMC.loc[0, 'interval'] = bw_all_DMC.loc[0, 'timestamp']

    bw_all_DMC['memory bandwidth (GB/s)'] = (bw_all_DMC['value'] * 64) / (bw_all_DMC['interval'] * 1000000000)

    return bw_all_DMC['memory bandwidth (GB/s)'][0]


def watch_point_avg(perf_raw_data_WC, up_or_down = "down"):
    perf_agg_data_WC = perf_raw_data_WC.groupby(['event']).agg(value=('value', 'sum'), timestamp = ('timestamp', 'max')).reset_index()
    perf_agg_data_WC = perf_agg_data_WC[perf_agg_data_WC["event"].str.contains("watchpoint_" + up_or_down)]
    perf_agg_data_WC = perf_agg_data_WC.drop('event', axis=1)

    bw_all_WC = perf_agg_data_WC.groupby(['timestamp']).agg(value=('value', 'sum')).reset_index()

    bw_all_WC['interval'] = bw_all_WC['timestamp'].diff()
    bw_all_WC.loc[0, 'interval'] = bw_all_WC.loc[0, 'timestamp']

    bw_all_WC['memory bandwidth (GB/s)'] = (bw_all_WC['value'] * 32) / (bw_all_WC['interval'] * 1000000000)

    return bw_all_WC['memory bandwidth (GB/s)'][0]

In [22]:
import os

repoPath = "/home/eerqi/Workplace/mem_bw_exp/repo/"

DMC_csv_dir = repoPath + "data/csv/dmc_620/"
WP_csv_dir = repoPath + "data/csv/watch_point/"

DMC_files = os.listdir(DMC_csv_dir)
WP_files = os.listdir(WP_csv_dir)

# DMC_bench = [file for file in DMC_files if file.startswith("500")]
# WP_bench = [file for file in WP_files if file.startswith("500")]

benches = list(set([fileName.split('.')[0] for fileName in DMC_files]))
benches.sort()

methods = [{"data_path": WP_csv_dir, "func": watch_point_avg, "name": "Watch_Point"},
           {"data_path": DMC_csv_dir, "func": dmc_620_avg, "name": "DMC_620"}]

chart = pd.DataFrame(columns=["DMC_620", "Watch_Point"], index=benches)

for method in methods:
    path, func, name = method["data_path"], method["func"], method["name"]
    files = os.listdir(path)
    for bench in benches:
        bench_files = [file for file in files if file.startswith(bench)]
        datas = []
        for file in bench_files:
            perf_raw_data = pd.read_csv(path + file,
                                        sep='|',
                                        header=None, 
                                        names=["timestamp", "value", "event"], 
                                        usecols=[0, 1, 3])
            perf_agg_data = perf_raw_data.groupby(['event']).agg(value=('value', 'sum'), timestamp = ('timestamp', 'max')).reset_index()
            datas.append(perf_agg_data)

        df_concatenated = pd.concat(datas, axis=0)

        bench_perf_data = df_concatenated.groupby(['event']).agg(value=('value', 'sum'), timestamp = ('timestamp', 'sum')).reset_index()

        avg = func(bench_perf_data)
        # print(f"bench: {bench}  method: {name}  average: {avg}")
        chart[name][bench] = avg
        
print(chart)

/tmp/ipykernel_914532/3759708643.py:43: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  chart[name][bench] = avg
/tmp/ipykernel_914532/3759708643.py:43: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are settin

       DMC_620 Watch_Point
500   6.062339    6.055601
502  56.897504   56.911936
503  41.262687   41.058328
505  26.425043   26.036345
507   51.60967   51.387079
508   9.647283    9.599729
510  16.653975   16.609752
511   0.061785     0.03492
519  126.94981  126.691764
520  47.210226   47.184302
521  88.443591   88.253929
523   23.16938   23.154626
525  12.481987   12.464113
526   3.360961    3.356693
527  78.504163   78.458037
531  15.643824   15.657096
538   2.671251    2.615501
541   4.450855    4.534289
544   2.036642    2.019702
548   0.048058    0.026635
549  93.140263   92.898352
554  55.753732   55.820721
557  38.667906   38.842655


/tmp/ipykernel_914532/3759708643.py:43: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  chart[name][bench] = avg
/tmp/ipykernel_914532/3759708643.py:43: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are settin

In [38]:
chart["WP_rel_err"] = (abs((chart["Watch_Point"] - chart["DMC_620"] )/ chart["DMC_620"]) * 100)
chart['WP_rel_err'] = chart['WP_rel_err'].apply(lambda x: f"{x:.2f}%")
# chart["CMN_MC_rel_err"] = (abs((chart["CMN_MC"] - chart["DMC_620"] )/ chart["DMC_620"]) * 100)
# chart['CMN_MC_rel_err'] = chart['CMN_MC_rel_err'].apply(lambda x: f"{x:.2f}%")
chart["WP_abs_err"] = abs((chart["Watch_Point"] - chart["DMC_620"] ))
# chart["CMN_MC_abs_err"] = abs((chart["CMN_MC"] - chart["DMC_620"] ))
print(chart)


       DMC_620 Watch_Point WP_rel_err WP_abs_err
500   6.062339    6.055601      0.11%   0.006738
502  56.897504   56.911936      0.03%   0.014432
503  41.262687   41.058328      0.50%   0.204359
505  26.425043   26.036345      1.47%   0.388698
507   51.60967   51.387079      0.43%   0.222591
508   9.647283    9.599729      0.49%   0.047554
510  16.653975   16.609752      0.27%   0.044222
511   0.061785     0.03492     43.48%   0.026865
519  126.94981  126.691764      0.20%   0.258046
520  47.210226   47.184302      0.05%   0.025924
521  88.443591   88.253929      0.21%   0.189662
523   23.16938   23.154626      0.06%   0.014754
525  12.481987   12.464113      0.14%   0.017874
526   3.360961    3.356693      0.13%   0.004269
527  78.504163   78.458037      0.06%   0.046127
531  15.643824   15.657096      0.08%   0.013272
538   2.671251    2.615501      2.09%   0.055749
541   4.450855    4.534289      1.87%   0.083434
544   2.036642    2.019702      0.83%    0.01694
548   0.048058    0.

In [39]:
chart.to_excel('average_bw_write_and_down.xlsx')